#### Import all libraries

In [1]:
from __future__ import print_function
import os
import collections
import SimpleITK as sitk
import numpy as np
import six
import radiomics
from radiomics import firstorder, glcm, imageoperations, shape, glrlm, glszm, getTestCase
import sys
import logging as logger
import time
import json
import os
import logging.config
import pandas as pd
import gc
import math

#### Functions to be used

In [2]:
def max_min_glszm_window():
    
    df_min = my_df.min(axis=0).to_frame()
    df_min = df_min.rename(index = str, columns={0: "Minimum"})

    df_max = my_df.max(axis=0).to_frame()
    df_max = df_max.rename(index = str, columns={0: "Maximum"})

    df_max_min = df_min.merge(df_max, how='outer', left_index=True, right_index=True)
    
    name_min_max_csv = 'glszm_img_{}_w{}_min_max.csv'.format(num_img, window_size)
    df_max_min.to_csv(name_min_max_csv, index=True, header=True)
    
    del df_min, df_max, df_max_min

In [3]:
def glszm_calculations():
    "compute all the GLSZM related parameters"
    
    import radiomics
    from radiomics import firstorder, glcm, imageoperations, shape, glrlm, glszm, getTestCase
    #calculate the original glszm
    glszmFeatures = glszm.RadiomicsGLSZM(croppedImage, croppedMask, **settings)
    glszmFeatures.enableAllFeatures()
    result = glszmFeatures.execute()

    print("Calculated GLSZM.....")

    glszm_org = glszmFeatures.P_glszm
    #The graylevels in original ROI after discretization
    graylevels_org = glszmFeatures.coefficients['grayLevels']

    #vector of all the gray-level valus present in the original ROI
    im_exp = sitk.GetArrayFromImage(croppedImage)
    iv_org =  np.unique(im_exp)#glszmFeatures.coefficients['ivector']
    iv_org = iv_org.astype(float)
    #print(str(iv_org))
    iv_org = iv_org + 1
    #print(str(iv_org))

    #vector of all the size zones in the original GLSZM
    jv_org = glszmFeatures.coefficients['jvector']

    # sum of all the size zones for all the graylevels of a particular size zones
    # for every column of GLSZM, add all the rows
    ps_org = glszmFeatures.coefficients['ps']

    #Remove the mineral or saturated phase if it exists FROM GLSZM
    if iv_org[iv_org.size - 1] == 256:
        print('Mineral phase is present because the graylevel of', iv_org[iv_org.size-1], 'exists.')
        mineralGrayLevels_idx = iv_org.size-1
        #Calculate the new GLSZM
        glszm_no_mineral = np.delete(glszm_org, mineralGrayLevels_idx, 1)
        s_nm = glszm_no_mineral.shape

        #Calculate the new ivector
        iv_new = iv_org[0:iv_org.size-1]  # shape (Ng,)

        #Calculate the new ps array with zero values
        glszm_2d_nm = glszm_no_mineral[0, 0:s_nm[1], 0:s_nm[2]]
        ps_new_zeros = np.sum(glszm_2d_nm, 0)
        ps_reshape = np.reshape(ps_new_zeros, (1,s_nm[2]))

        #Delete columns that specify zone sizes not present in the ROI
        emptyZoneSizes = np.where(np.sum(ps_reshape, 0) == 0)
        glszm_new = np.delete(glszm_no_mineral, emptyZoneSizes, 2)
        s_new = glszm_new.shape
        jv_new = np.delete(jv_org, emptyZoneSizes)
        ps_new = np.delete(ps_org, emptyZoneSizes, 1)

        # Get the number of zones in this GLSZM
        Nz_new = np.sum(glszm_new, (1, 2))  # shape (Nvox,)
        Nz_new[Nz_new == 0] = 1  # set sum to numpy.spacing(1) if sum is 0?

        # Get the number of voxels represented by this GLSZM: Multiply the zones by their size and sum them
        Np_new = np.sum(ps_new * jv_new[None, :], 1)  # shape (Nvox, )
        Np_new[Np_new == 0] = 1

        # Get the pg
        glszm_2d_new = glszm_new[0, 0:s_new[1], 0:s_new[2]]
        pg_reshape = np.sum(glszm_2d_new, 1)
        pg_new = np.reshape(pg_reshape, (1,s_new[1]))

    else:

        #just crab the coeffiecients as usual
        iv_new = iv_org
        jv_new = jv_org
        glszm_new = glszm_org
        Nz_new = glszmFeatures.coefficients['Nz']
        Np_new = glszmFeatures.coefficients['Np']
        pg_new = glszmFeatures.coefficients['pg']
        ps_new = glszmFeatures.coefficients['ps']

    jvector = jv_new
    ivector = iv_new
    ps = ps_new
    pg = pg_new
    Np = Np_new
    Nz = Nz_new
    pg_norm = pg/Nz[:, None]
    ps_norm = ps/Nz[:, None]
    eps = np.spacing(1)
    glszm = glszm_new
    print('The shape of the old glszm is', str(glszm_org.shape), '. The shape of the new glszm is', str(glszm.shape))
    glszm_norm = glszm_new/Nz[:, None, None]

    print("GLSZM processing complete ...")

    #del glszm_org, glszm_new, iv_org, jv_org, ps_org, mineralGrayLevels_idx, glszm_no_mineral, s_nm, iv_new, glszm_2d_nm
    #del ps_new_zeros, ps_reshape, emptyZoneSizes, glszm_new, s_new, jv_new, ps_new, Nz_new, Np_new, glszm_2d_nm, pg_reshape
    #del pg_new

    #Calculate glszm features and normalize the features
    print("Initiating GLSZM feature calculation...... ")

    sae = np.sum(ps / (jvector[None, :] ** 2), 1) / Nz #small area emphaisis
    sae = sae.astype(float)
    lae = np.sum(ps * (jvector[None, :] ** 2), 1) / Nz #large area emphaisis
    iv = np.sum(pg ** 2, 1) / Nz #graylevel non-uniformity
    ivn = np.sum(pg ** 2, 1) / Nz ** 2 #graylevel non-uniformity normalized
    szv = np.sum(ps ** 2, 1) / Nz #size zone uniformity
    szvn = np.sum(ps ** 2, 1) / Nz ** 2 #size zone uniformity normalised
    zp = Nz / Np # zone percentage
    u_i = np.sum(pg_norm * ivector[None, :], 1, keepdims=True) # mean calculation for graylevel variance
    glv = np.sum(pg_norm * (ivector[None, :] - u_i) ** 2, 1) # graylevel variance
    u_j = np.sum(ps_norm * jvector[None, :], 1, keepdims=True) #mean caluclation of zone variance
    zv = np.sum(ps_norm * (jvector[None, :] - u_j) ** 2, 1) #zone variance
    ze = -np.sum(glszm_norm * np.log2(glszm_norm + eps), (1, 2)) #zone entropy
    lie = np.sum(pg / (ivector[None, :] ** 2), 1) / Nz #Low graylevel zone emphaisis
    hie = np.sum(pg * (ivector[None, :] ** 2), 1) / Nz #high graylevel zone emphaisis
    lisae = np.sum(glszm / ((ivector[None, :, None] ** 2) * (jvector[None, None, :] ** 2)), (1, 2)) / Nz #small area low graylevel emphaisis
    hisae = np.sum(glszm * (ivector[None, :, None] ** 2) / (jvector[None, None, :] ** 2), (1, 2)) / Nz #small area high graylevel emphaisis
    lilae = np.sum(glszm * (jvector[None, None, :] ** 2) / (ivector[None, :, None] ** 2), (1, 2)) / Nz #large area low gray level emphaisis
    hilae = np.sum(glszm * (ivector[None, :, None] ** 2) * (jvector[None, None, :] ** 2), (1, 2)) / Nz #large area high gray level emphaisis

    #normalise features by the number of voxels
    iv_np = iv/Np
    glv_np = glv/Np
    hie_np = hie/Np
    lae_np = lae/Np
    hilae_np = hilae/Np
    lilae_np = lilae/Np
    szv_np = szv/Np
    hisae_np = hisae/Np
    zv_np = zv/Np

    #append values
    v1.append(iv[0])
    v1_norm.append(iv_np[0])
    v2.append(ivn[0])
    v3.append(glv[0])
    v3_norm.append(glv_np[0])
    v4.append(hie[0])
    v4_norm.append(hie_np[0])
    v5.append(lae[0])
    v5_norm.append(lae_np[0])
    v6.append(hilae[0])
    v6_norm.append(hilae_np[0])
    v7.append(lilae[0])
    v7_norm.append(lilae_np[0])
    v8.append(lie[0])
    v9.append(szv[0])
    v9_norm.append(szv_np[0])
    v10.append(szvn[0])
    v11.append(sae[0])
    v12.append(hisae[0])
    v12_norm.append(hisae_np[0])
    v13.append(lisae[0])
    v14.append(ze[0])
    v15.append(zp[0])
    v16.append(zv[0])
    v16_norm.append(zv_np[0])

    del sae, lae, iv, ivn, szv, szvn, zp, u_i, glv, u_j, zv, ze, lie, hie, lisae, hisae, lilae, hilae
    del iv_np, glv_np, hie_np, lae_np, hilae_np, lilae_np, szv_np, hisae_np, zv_np

##### Window calculations for upto half the window_size

In [4]:
%%time
print("Starting the automation loop for GLSZM feature extraction")

# All the greyscale images of various porous media types are named as data#.nrrd. 
# Iterate over the names to calculate the GLSZM features
for num_img in range(5, 6):
    
    # Create the variables for image and label
    global image_1, label_1

    # Path of the images used in REV analysis
    path ='D:\\Ankita PhD\\pyradiomics\\data\\data{}.nrrd'.format(num_img)
    
    # Read the nrrd image using the SITK library. Nrrd images are 8-bit images with intensity values between 0 to 255
    image_1 = sitk.ReadImage(path) 
    
    # Set the settings paramter needed  for the bin_width and label
    # Bin_Width = 1 (as I don't want any further reduction in greyscale values and want to preserve the information)
    # label is a nrrd image that incorporates the window size and hence, gives a ROI for the GLSZM calculation.
    # The minimum size is set by the user and the mavimum size is the size of the image
    settings = {}
    settings['label'] = 255
    settings['binWidth'] = 1

    # Set-up logging for all the calculations
    log_file = 'D:\\Ankita PhD\\pyradiomics\\log_messages\\log{}.txt'.format(num_img)
    handler = logging.FileHandler(filename=log_file, mode='w')  # overwrites log_files from previous runs. Change mode to 'a' to append.
    formatter = logging.Formatter("%(levelname)s:%(name)s: %(message)s")  # format string for log messages
    handler.setFormatter(formatter)
    radiomics.logger.addHandler(handler)

    # Control the amount of logging stored by setting the level of the logger. N.B. if the level is higher than the
    # Verbositiy level, the logger level will also determine the amount of information printed to the output
    radiomics.logger.setLevel(logging.DEBUG)

    # the size of the mask (same as label) should be the same as that of the image
    mask_x = image_1.GetWidth()
    mask_y = image_1.GetWidth()
    mask_z = image_1.GetWidth()

    print("Initializing window calculcations for image",str(num_img))
    
    # here the window_size refers to the region of mask that will be used to segment the ROI from the image. The window
    # sizes decreases with every iteration
    u_window_size = [100, 150, 200, 250, 300, 350] 
    global window_size, total_iter, expect_iter, i_loc, j_loc, z_loc, bb, correctedMask, croppedImage, croppedMask, glcm_window
    global v1, v2, v3, v4, v5, v6, v7, v8, v9, v10, v11, v12, v13, v14, v15, v16
    global v1_norm, v3_norm, v4_norm, v5_norm, v6_norm, v7_norm, v9_norm, v12_norm, v16_norm
    
    for window_size in u_window_size:  
       
        print("Starting the window calculcation for image",str(num_img),"for window size = ", str(window_size)) 
        total_iter = 0

        v1 = []    
        v2 = []
        v3 = []
        v4 = []
        v5 = []
        v6 = []
        v7 = []
        v8 = []
        v9 = []
        v10 = []
        v11 = []
        v12 = []
        v13 = []
        v14 = []
        v15 = []
        v16 = []
        v1_norm = []
        v3_norm = []
        v4_norm = []
        v5_norm = []
        v6_norm = []
        v7_norm = []
        v9_norm = []
        v12_norm = []
        v16_norm = []

        step_x = window_size-1
        step_y = window_size-1
        step_z = window_size-1

        i_loc=0
        j_loc=0
        z_loc=0
        i_loc_max = window_size
        j_loc_max = window_size
        z_loc_max = window_size
        
        
        # x-direction
        while i_loc_max <= mask_x and total_iter < 1000:
            
            #y-direction
            j_loc_max = window_size
            while j_loc_max <= mask_y and total_iter < 1000:
                
                #z-direction
                z_loc_max = window_size
                while z_loc_max <= mask_z and total_iter < 1000:
                    
                    full_mask = np.zeros((mask_x, mask_y, mask_z), dtype = np.uint8)
                    full_mask[i_loc:i_loc_max,j_loc:j_loc_max,z_loc:z_loc_max] = 255
                    print(str(full_mask.size))
                    
                    label_1 = sitk.GetImageFromArray(full_mask, isVector=False)
                    del full_mask

                    bb, correctedMask = imageoperations.checkMask(image_1, label_1, label=255)
                    if correctedMask is not None:
                        label_1 = correctedMask
                    croppedImage, croppedMask = imageoperations.cropToTumorMask(image_1, label_1, bb)
                    del correctedMask, label_1

                    glszm_calculations()
                    
                    z_loc = z_loc + step_z
                    total_iter = total_iter + 1
                    print(total_iter)
                    z_loc_max = z_loc+window_size

                z_loc = 0
                j_loc = j_loc + step_y
                j_loc_max = j_loc+window_size

            j_loc = 0
            i_loc = i_loc + step_x
            i_loc_max = i_loc+window_size
        
        print("Completed Calculations for Window ", str(window_size), ". The number of iterations is ", str(total_iter)) 
        
        w1 = [window_size] * total_iter
        # Create a data-frame to store all the GLSZM features for the image        
        my_df = pd.DataFrame(list(zip(w1, v1, v2, v3, v4, v5, v6, v7, v8, v9, v10, v11, v12, v13, v14, v15, v16, \
                                  v1_norm, v3_norm, v4_norm, v5_norm, v6_norm, v7_norm, v9_norm, v12_norm, v16_norm)))
        my_df = my_df.rename(index = str, columns={0: "Window Sizes", 1: "GrayLevelNonUniformity", 2: "GrayLevelNonUniformityNormalized", \
                                   3: "GrayLevelVariance", 4: "HighGrayLevelZoneEmphasis", 5: "LargeAreaEmphasis", \
                                   6: "LargeAreaHighGrayLevelEmphasis", 7: "LargeAreaLowGrayLevelEmphasis", 8: "LowGrayLevelZoneEmphasis", \
                                   9: "SizeZoneNonUniformity", 10: "SizeZoneNonUniformityNormalized", 11: "SmallAreaEmphasis", \
                                   12: "SmallAreaHighGrayLevelEmphasis", 13: "SmallAreaLowGrayLevelEmphasis", 14: "ZoneEntropy", \
                                   15: "ZonePercentage", 16: "ZoneVariance", 17: "Norm GrayLevelNonUniformity", \
                                   18: "Norm GrayLevelVariance", 19: "Norm HighGrayLevelZoneEmphasis", 20: "Norm LargeAreaEmphasis", \
                                   21: "Norm LargeAreaHighGrayLevelEmphasis", 22: "Norm LargeAreaLowGrayLevelEmphasis", \
                                   23: "Norm SizeZoneNonUniformity", 24:"Norm SmallAreaHighGrayLevelEmphasis", 25: "Norm ZoneVariance"})    
       
        
        name_csv = 'glszm_img_{}_w{}.csv'.format(num_img, window_size)
        my_df.to_csv(name_csv, index=False, header=True)
        
        max_min_glszm_window()
        
        print("Saved glcm values of window size ", str(window_size))
        del my_df, name_csv 

Starting the automation loop for GLSZM feature extraction
Initializing window calculcations for image 5
Starting the window calculcation for image 5 for window size =  100
343000000
Calculated GLSZM.....
The shape of the old glszm is (1, 215, 33) . The shape of the new glszm is (1, 215, 33)
GLSZM processing complete ...
Initiating GLSZM feature calculation...... 
1
343000000
Calculated GLSZM.....
Mineral phase is present because the graylevel of 256.0 exists.
The shape of the old glszm is (1, 251, 36) . The shape of the new glszm is (1, 250, 36)
GLSZM processing complete ...
Initiating GLSZM feature calculation...... 
2
343000000
Calculated GLSZM.....
The shape of the old glszm is (1, 230, 37) . The shape of the new glszm is (1, 230, 37)
GLSZM processing complete ...
Initiating GLSZM feature calculation...... 
3
343000000
Calculated GLSZM.....
The shape of the old glszm is (1, 234, 37) . The shape of the new glszm is (1, 234, 37)
GLSZM processing complete ...
Initiating GLSZM feature c

Calculated GLSZM.....
The shape of the old glszm is (1, 196, 61) . The shape of the new glszm is (1, 196, 61)
GLSZM processing complete ...
Initiating GLSZM feature calculation...... 
33
343000000
Calculated GLSZM.....
Mineral phase is present because the graylevel of 256.0 exists.
The shape of the old glszm is (1, 246, 54) . The shape of the new glszm is (1, 245, 53)
GLSZM processing complete ...
Initiating GLSZM feature calculation...... 
34
343000000
Calculated GLSZM.....
Mineral phase is present because the graylevel of 256.0 exists.
The shape of the old glszm is (1, 233, 53) . The shape of the new glszm is (1, 232, 52)
GLSZM processing complete ...
Initiating GLSZM feature calculation...... 
35
343000000
Calculated GLSZM.....
Mineral phase is present because the graylevel of 256.0 exists.
The shape of the old glszm is (1, 240, 47) . The shape of the new glszm is (1, 239, 46)
GLSZM processing complete ...
Initiating GLSZM feature calculation...... 
36
343000000
Calculated GLSZM....

Calculated GLSZM.....
Mineral phase is present because the graylevel of 256.0 exists.
The shape of the old glszm is (1, 242, 49) . The shape of the new glszm is (1, 241, 47)
GLSZM processing complete ...
Initiating GLSZM feature calculation...... 
66
343000000
Calculated GLSZM.....
Mineral phase is present because the graylevel of 256.0 exists.
The shape of the old glszm is (1, 218, 53) . The shape of the new glszm is (1, 217, 52)
GLSZM processing complete ...
Initiating GLSZM feature calculation...... 
67
343000000
Calculated GLSZM.....
Mineral phase is present because the graylevel of 256.0 exists.
The shape of the old glszm is (1, 240, 51) . The shape of the new glszm is (1, 239, 50)
GLSZM processing complete ...
Initiating GLSZM feature calculation...... 
68
343000000
Calculated GLSZM.....
Mineral phase is present because the graylevel of 256.0 exists.
The shape of the old glszm is (1, 248, 47) . The shape of the new glszm is (1, 247, 46)
GLSZM processing complete ...
Initiating GL

Calculated GLSZM.....
Mineral phase is present because the graylevel of 256.0 exists.
The shape of the old glszm is (1, 253, 36) . The shape of the new glszm is (1, 252, 32)
GLSZM processing complete ...
Initiating GLSZM feature calculation...... 
99
343000000
Calculated GLSZM.....
Mineral phase is present because the graylevel of 256.0 exists.
The shape of the old glszm is (1, 244, 37) . The shape of the new glszm is (1, 243, 37)
GLSZM processing complete ...
Initiating GLSZM feature calculation...... 
100
343000000
Calculated GLSZM.....
Mineral phase is present because the graylevel of 256.0 exists.
The shape of the old glszm is (1, 248, 37) . The shape of the new glszm is (1, 247, 37)
GLSZM processing complete ...
Initiating GLSZM feature calculation...... 
101
343000000
Calculated GLSZM.....
Mineral phase is present because the graylevel of 256.0 exists.
The shape of the old glszm is (1, 245, 40) . The shape of the new glszm is (1, 244, 40)
GLSZM processing complete ...
Initiating 

Calculated GLSZM.....
Mineral phase is present because the graylevel of 256.0 exists.
The shape of the old glszm is (1, 235, 57) . The shape of the new glszm is (1, 234, 57)
GLSZM processing complete ...
Initiating GLSZM feature calculation...... 
131
343000000
Calculated GLSZM.....
Mineral phase is present because the graylevel of 256.0 exists.
The shape of the old glszm is (1, 238, 60) . The shape of the new glszm is (1, 237, 58)
GLSZM processing complete ...
Initiating GLSZM feature calculation...... 
132
343000000
Calculated GLSZM.....
The shape of the old glszm is (1, 193, 67) . The shape of the new glszm is (1, 193, 67)
GLSZM processing complete ...
Initiating GLSZM feature calculation...... 
133
343000000
Calculated GLSZM.....
Mineral phase is present because the graylevel of 256.0 exists.
The shape of the old glszm is (1, 239, 60) . The shape of the new glszm is (1, 238, 57)
GLSZM processing complete ...
Initiating GLSZM feature calculation...... 
134
343000000
Calculated GLSZM

KeyboardInterrupt: 

##### Window calculations for after half the window size

In [5]:
%%time
print("Starting the automation loop for GLSZM feature extraction")

# All the greyscale images of various porous media types are named as data#.nrrd. 
# Iterate over the names to calculate the GLSZM features
for num_img in range(5, 6):
    
    # Create the variables for image and label
    global image_1, label_1

    # Path of the images used in REV analysis
    path ='D:\\Ankita PhD\\pyradiomics\\data\\data{}.nrrd'.format(num_img)
    
    # Read the nrrd image using the SITK library. Nrrd images are 8-bit images with intensity values between 0 to 255
    image_1 = sitk.ReadImage(path) 
    
    # Set the settings paramter needed for the bin_width and label
    # Bin_Width = 1 (as I don't want any further reduction in greyscale values and want to preserve the information)
    # label is a nrrd image that incorporates the window size and hence, gives a ROI for the GLSZM calculation.
    # The minimum size is set by the user and the mavimum size is the size of the image
    settings = {}
    settings['label'] = 255
    settings['binWidth'] = 1

    # Set-up logging for all the calculations
    log_file = 'D:\\Ankita PhD\\pyradiomics\\log_messages\\log{}.txt'.format(num_img)
    handler = logging.FileHandler(filename=log_file, mode='w')  # overwrites log_files from previous runs. Change mode to 'a' to append.
    formatter = logging.Formatter("%(levelname)s:%(name)s: %(message)s")  # format string for log messages
    handler.setFormatter(formatter)
    radiomics.logger.addHandler(handler)

    # Control the amount of logging stored by setting the level of the logger. N.B. if the level is higher than the
    # Verbositiy level, the logger level will also determine the amount of information printed to the output
    radiomics.logger.setLevel(logging.DEBUG)

    # the size of the mask (same as label) should be the same as that of the image
    mask_x = image_1.GetWidth()
    mask_y = image_1.GetWidth()
    mask_z = image_1.GetWidth()

    print("Initializing window calculcations for image",str(num_img))
    
    # here the window_size refers to the region of mask that will be used to segment the ROI from the image. The window
    # sizes decreases with every iteration
    u_window_size = [400, 450, 500, 550, 600]
    global window_size, total_iter, expect_iter, i_loc, j_loc, z_loc, bb, correctedMask, croppedImage, croppedMask, glcm_window
    global my_df, df_min, df_max, df_max_min
    
    for window_size in u_window_size:  
       
        print("Starting the window calculcation for image",str(num_img),"for window size = ", str(window_size)) 
        total_iter = 0
        v1 = []
        v2 = []
        v3 = []
        v4 = []
        v5 = []
        v6 = []
        v7 = []
        v8 = []
        v9 = []
        v10 = []
        v11 = []
        v12 = []
        v13 = []
        v14 = []
        v15 = []
        v16 = []
        v1_norm = []
        v3_norm = []
        v4_norm = []
        v5_norm = []
        v6_norm = []
        v7_norm = []
        v9_norm = []
        v12_norm = []
        v16_norm = []

        step_x = window_size-1
        step_y = window_size-1
        step_z = window_size-1

        i_loc=0
        j_loc=0
        z_loc=0
        i_loc_max = window_size
        j_loc_max = window_size
        z_loc_max = window_size

        # Position 1
        total_iter = total_iter + 1
        full_mask = np.zeros((mask_x, mask_y, mask_z), dtype = np.uint8)
        full_mask[i_loc:i_loc_max,j_loc:j_loc_max,z_loc:z_loc_max] = 255
        print(str(full_mask.size))

        label_1 = sitk.GetImageFromArray(full_mask, isVector=False)
        del full_mask

        bb, correctedMask = imageoperations.checkMask(image_1, label_1, label=255)
        if correctedMask is not None:
            label_1 = correctedMask
        croppedImage, croppedMask = imageoperations.cropToTumorMask(image_1, label_1, bb)
        del correctedMask, label_1

        glszm_calculations()
        
        #Position 2
        total_iter = total_iter + 1
        full_mask = np.zeros((mask_x, mask_y, mask_z), dtype = np.uint8)
        full_mask[mask_x-window_size+1:mask_x, mask_y-window_size+1:mask_y, mask_z-window_size+1:mask_z] = 255
        
        
        label_1 = sitk.GetImageFromArray(full_mask, isVector=False)
        del full_mask

        bb, correctedMask = imageoperations.checkMask(image_1, label_1, label=255)
        if correctedMask is not None:
            label_1 = correctedMask
        croppedImage, croppedMask = imageoperations.cropToTumorMask(image_1, label_1, bb)
        del correctedMask, label_1

        glszm_calculations()
        
        #Position 3
        total_iter = total_iter + 1
        full_mask = np.zeros((mask_x, mask_y, mask_z), dtype = np.uint8)
        full_mask[i_loc:i_loc_max, mask_y-window_size+1:mask_y, z_loc:z_loc_max] = 255
        
        
        label_1 = sitk.GetImageFromArray(full_mask, isVector=False)
        del full_mask

        bb, correctedMask = imageoperations.checkMask(image_1, label_1, label=255)
        if correctedMask is not None:
            label_1 = correctedMask
        croppedImage, croppedMask = imageoperations.cropToTumorMask(image_1, label_1, bb)
        del correctedMask, label_1
        
        glszm_calculations()

        #Position 4
        total_iter = total_iter + 1
        full_mask = np.zeros((mask_x, mask_y, mask_z), dtype = np.uint8)
        full_mask[mask_x-window_size+1:mask_x, mask_y-window_size+1:mask_y, z_loc:z_loc_max] = 255
        
        
        label_1 = sitk.GetImageFromArray(full_mask, isVector=False)
        del full_mask

        bb, correctedMask = imageoperations.checkMask(image_1, label_1, label=255)
        if correctedMask is not None:
            label_1 = correctedMask
        croppedImage, croppedMask = imageoperations.cropToTumorMask(image_1, label_1, bb)
        del correctedMask, label_1

        glszm_calculations()
        
        #Position 5
        total_iter = total_iter + 1
        full_mask = np.zeros((mask_x, mask_y, mask_z), dtype = np.uint8)
        full_mask[i_loc:i_loc_max, j_loc:j_loc_max, mask_z-window_size+1:mask_z] = 255
    
        label_1 = sitk.GetImageFromArray(full_mask, isVector=False)
        del full_mask

        bb, correctedMask = imageoperations.checkMask(image_1, label_1, label=255)
        if correctedMask is not None:
            label_1 = correctedMask
        croppedImage, croppedMask = imageoperations.cropToTumorMask(image_1, label_1, bb)
        del correctedMask, label_1

        glszm_calculations()
        
        #Position 6
        total_iter = total_iter + 1
        full_mask = np.zeros((mask_x, mask_y, mask_z), dtype = np.uint8)
        full_mask[mask_x-window_size+1:mask_x, j_loc:j_loc_max, mask_z-window_size+1:mask_z] = 255
        
        label_1 = sitk.GetImageFromArray(full_mask, isVector=False)
        del full_mask

        bb, correctedMask = imageoperations.checkMask(image_1, label_1, label=255)
        if correctedMask is not None:
            label_1 = correctedMask
        croppedImage, croppedMask = imageoperations.cropToTumorMask(image_1, label_1, bb)
        del correctedMask, label_1
        
        glszm_calculations()
        
        #Position 7
        total_iter = total_iter + 1
        full_mask = np.zeros((mask_x, mask_y, mask_z), dtype = np.uint8)
        full_mask[i_loc:i_loc_max, mask_y-window_size+1:mask_y, mask_z-window_size+1:mask_z] = 255
        
        label_1 = sitk.GetImageFromArray(full_mask, isVector=False)
        del full_mask

        bb, correctedMask = imageoperations.checkMask(image_1, label_1, label=255)
        if correctedMask is not None:
            label_1 = correctedMask
        croppedImage, croppedMask = imageoperations.cropToTumorMask(image_1, label_1, bb)
        del correctedMask, label_1
        
        glszm_calculations()
        
        #Position 8
        total_iter = total_iter + 1
        full_mask = np.zeros((mask_x, mask_y, mask_z), dtype = np.uint8)
        full_mask[mask_x-window_size+1:mask_x, j_loc:j_loc_max, z_loc:z_loc_max] = 255
        
        
        label_1 = sitk.GetImageFromArray(full_mask, isVector=False)
        del full_mask

        bb, correctedMask = imageoperations.checkMask(image_1, label_1, label=255)
        if correctedMask is not None:
            label_1 = correctedMask
        croppedImage, croppedMask = imageoperations.cropToTumorMask(image_1, label_1, bb)
        del correctedMask, label_1

        glszm_calculations()
        
        print("Completed Calculations for Window ", str(window_size), ". The number of iterations is ", str(total_iter)) 
        
        w1 = [window_size] * total_iter
        # Create a data-frame to store all the GLSZM features for the image         
        my_df = pd.DataFrame(list(zip(w1, v1, v2, v3, v4, v5, v6, v7, v8, v9, v10, v11, v12, v13, v14, v15, v16, \
                                  v1_norm, v3_norm, v4_norm, v5_norm, v6_norm, v7_norm, v9_norm, v12_norm, v16_norm)))
        my_df = my_df.rename(index = str, columns={0: "Window Sizes", 1: "GrayLevelNonUniformity", 2: "GrayLevelNonUniformityNormalized", \
                                   3: "GrayLevelVariance", 4: "HighGrayLevelZoneEmphasis", 5: "LargeAreaEmphasis", \
                                   6: "LargeAreaHighGrayLevelEmphasis", 7: "LargeAreaLowGrayLevelEmphasis", 8: "LowGrayLevelZoneEmphasis", \
                                   9: "SizeZoneNonUniformity", 10: "SizeZoneNonUniformityNormalized", 11: "SmallAreaEmphasis", \
                                   12: "SmallAreaHighGrayLevelEmphasis", 13: "SmallAreaLowGrayLevelEmphasis", 14: "ZoneEntropy", \
                                   15: "ZonePercentage", 16: "ZoneVariance", 17: "Norm GrayLevelNonUniformity", \
                                   18: "Norm GrayLevelVariance", 19: "Norm HighGrayLevelZoneEmphasis", 20: "Norm LargeAreaEmphasis", \
                                   21: "Norm LargeAreaHighGrayLevelEmphasis", 22: "Norm LargeAreaLowGrayLevelEmphasis", \
                                   23: "Norm SizeZoneNonUniformity", 24:"Norm SmallAreaHighGrayLevelEmphasis", 25: "Norm ZoneVariance"})    
       
        name_csv = 'glszm_img_{}_w{}.csv'.format(num_img, window_size)
        my_df.to_csv(name_csv, index=False, header=True)
        
        max_min_glszm_window()
        
        print("Saved glcm values of window size ", str(window_size))
        del my_df, name_csv 

Starting the automation loop for GLSZM feature extraction
Initializing window calculcations for image 5
Starting the window calculcation for image 5 for window size =  400
343000000


KeyboardInterrupt: 